# MHKiT Metocean Data

Metocean data is a useful addition to wave and current ocean data for the design for offshore structures, including wave energy converters (WECs). Metocean data primarily includes information pertaining to the wind, wave, and climate conditions on the ocean surface at a given location.

MHKiT includes functions to request and manipulate metocean data from the National Data Buoy Center, including spectral wave density, standard meteorological data and continuous wind speed. MHKiT also includes function to request and manipulate metocean data from the WIND Toolkit, including vertical profiles of wind speed, wind direction, precipitation, temperature, and pressure, as well as precipitation rate and relative humidity.

As a demonstration, this notebook will walk through the following steps to compare the surface wind speed and direction at a given location using both NDBC and WIND Toolkit data, and then plot a vertical air temperature profile.

 1. Request Continous Wind Data from NDBC
 2. Request Surface Wind Data from WIND Toolkit
 3. Compare NDBC and WIND Toolkit Data
 4. Request Temperature Data from WIND Toolkit
 5. Visualize Temperature Data
 
 TODO remove:
     - Calculate and plot the hourly mean wind speeds for a given date
     - Plot a wind rose of the continuous speeds for a given date
     - Plot vertical temperature profile for a given datetime

We will start by importing the necessary python packages (`pandas`, `numpy`, `matplotlib`), and MHKiT submodule (`wave.io`).

In [1]:
from mhkit.wave.io import ndbc
from mhkit.wave.io import wind_toolkit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Request Continuous Wind Data from NDBC
   
MHKiT can be used to request historical data from the National Data Buoy Center ([NDBC](https://www.ndbc.noaa.gov/)). This process is split into the following steps:

- Query available NDBC data  
- Select years of interest 
- Request data from NDBC
- Convert the DataFrames to DateTime Index
 

### Query available NDBC data  
Looking at the help for the `ndbc.available_data` function (`help(ndbc.available_data)`) the function requires a parameter to be specified and optionally the user may provide a station ID as a string. A full list of available historical parameters can be found [here](https://www.ndbc.noaa.gov/data/historical/) although only some of these are currently supported.  We are interested in historical continuous wind data `'cwind'`. Additionally, we will specify the buoy number as `'46022'` to only return data associated with this site.

In [2]:
# Specify the parameter as continuous wind speeds and the buoy number to be 46022
ndbc_dict = {'parameter':'cwind','buoy_number':'46022'} 
available_data = ndbc.available_data(ndbc_dict['parameter'], ndbc_dict['buoy_number'])
available_data

,id,year,filename
1211,46022,1996,46022c1996.txt.gz
1212,46022,1997,46022c1997.txt.gz
1213,46022,1998,46022c1998.txt.gz
1214,46022,1999,46022c1999.txt.gz
1215,46022,2000,46022c2000.txt.gz
1216,46022,2001,46022c2001.txt.gz
1217,46022,2002,46022c2002.txt.gz
1218,46022,2003,46022c2003.txt.gz
1219,46022,2004,46022c2004.txt.gz
1220,46022,2005,46022c2005.txt.gz


### Select years of interest

The `ndbc.available_data` function has returned a DataFrame with columns 'id', 'year', and 'filename'. The year column is of type int while the filename and id (5 digit alpha-numeric specifier) are of type string. In this case, the years returned from `available_data` span  1996 to the last year the buoy was operational (currently 2018 for 46022 wind data). For demonstration, we have decided we are interested in the data from 2018, so we will create a new `years_of_interest` DataFrame which only contains the year 2018.

In [3]:
# Slice the available data to only include 2018 and more recent
years_of_interest = available_data[available_data.year == 2018]
years_of_interest


,id,year,filename
1233,46022,2018,46022c2018.txt.gz


### Request Data from NDBC

The filename column in our `years_of_interest` DataFrame and the parameter is needed to request the data. To get the data we can use the `ndbc.request_data` function for each buoy id and year in the passed DataFrame. This function will return the parameter data as a dictionary of DataFrames which may be accessed by buoy id and then the year for multiple buoys or just the year for a single buoy. 

In [4]:
# Get dictionary of parameter data by year
ndbc_dict['filenames'] = years_of_interest['filename']
requested_data = ndbc.request_data(ndbc_dict['parameter'], ndbc_dict['filenames'])
requested_data

{'2018':         #YY  MM  DD  hh  mm  WDIR  WSPD  GDR   GST  GTIME
 0      2017  12  31  23   0   206   7.8  999  99.0   9999
 1      2017  12  31  23  10   202   8.3  999  99.0   9999
 2      2017  12  31  23  20   199   8.2  999  99.0   9999
 3      2017  12  31  23  30   194   7.6  999  99.0   9999
 4      2017  12  31  23  40   188   6.6  999  99.0   9999
 ...     ...  ..  ..  ..  ..   ...   ...  ...   ...    ...
 18319  2018   5   8  16  10   107   0.9  999  99.0   9999
 18320  2018   5   8  16  20   210   0.3  999  99.0   9999
 18321  2018   5   8  16  30   271   1.1  999  99.0   9999
 18322  2018   5   8  16  40   275   1.4  999  99.0   9999
 18323  2018   5   8  16  50   277   1.4  281   2.4   1642
 
 [18324 rows x 10 columns]}

### Convert the DataFrames to DateTime Index

The data returned for each year has a variable number of columns for the year, month, day, hour, minute, and the way the columns are formatted (this is a primary reason for return a dictionary of DataFrames indexed by years). A common step a user will want to take is to remove the inconsistent NDBC date/ time columns and create a standard DateTime index. The MHKiT function `ndbc.to_datetime_index` will perform this standardization by parsing the NDBC date/ time columns into DateTime format and setting this as the DataFrame Index and removing the NDBC date/ time columns. This function operates on a DateFrame therefore we will only pass the relevant year of the `ndbc_requested_data` dictionary.

In [5]:
# Convert the header dates to a Datetime Index and remove NOAA date columns for each year
ndbc_dict['2018'] = ndbc.to_datetime_index(ndbc_dict['parameter'], requested_data['2018'])

# Replace 99, 999, 9999 with NaN
ndbc_dict['2018'] = ndbc_dict['2018'].replace({99.0:np.NaN, 999:np.NaN, 9999:np.NaN})

# Display DataFrame of 46022 data from 2018
ndbc_dict['2018']

,WDIR,WSPD,GDR,GST,GTIME
date,,,,,
2017-12-31 23:00:00,206.0,7.8,NaN,NaN,NaN
2017-12-31 23:10:00,202.0,8.3,NaN,NaN,NaN
2017-12-31 23:20:00,199.0,8.2,NaN,NaN,NaN
2017-12-31 23:30:00,194.0,7.6,NaN,NaN,NaN
2017-12-31 23:40:00,188.0,6.6,NaN,NaN,NaN
...,...,...,...,...,...
2018-05-08 16:10:00,107.0,0.9,NaN,NaN,NaN
2018-05-08 16:20:00,210.0,0.3,NaN,NaN,NaN
2018-05-08 16:30:00,271.0,1.1,NaN,NaN,NaN


## 2. Request Surface Wind Data from WIND Toolkit

MHKiT can be used to request historical data from [WIND Toolkit](https://www.nrel.gov/grid/wind-toolkit.html). This process is split into the following steps:

- Choose input parameters 
- Request data from WIND Toolkit
 

### Choose input parameters
Looking at the help for `wind_toolkit.request_wtk_point_data` function (`help(wind_toolkit.request_wtk_point_data)`) the function requires four input parameters to be specified:
* time interval
* parameter
* location
* year

A full list of available historical parameters can be found [here](https://github.com/NREL/hsds-examples/blob/master/datasets/WINDToolkit.md#model).  We are interested in historical surface wind data `'windspeed_10m'`, `'winddirection_10m'`. Additionally, we will use the latitude/longitude location based on NDBC buoy `'46022'` to compare properly. The buoy location can be found [here](https://www.ndbc.noaa.gov/data/stations/station_table.txt).

In [6]:
# Input parameters for site of interest
wtk_inputs = {'time_interval':'1-hour',
              'parameter':['windspeed_10m','winddirection_10m'],
              'year':[2018],
              'lat_lon':(40.748, -124.527)}

### Request data from the WIND Toolkit
To get the data we can use the `wind_toolkit.request_wtk_point_data` function with multiple parameters, locations and years. If multiple locations are input, they must correspond to the same hindcast region. This function will return the parameter data as a DataFrame which may be accessed by parameter name and index of the location. Here we only use the inputs defined above, so the DataFrame key is `'wind_speed_10m_0'`. 

In [ ]:
wtk_data, wtk_metadata = wind_toolkit.request_wtk_point_data(
    wtk_inputs['time_interval'],wtk_inputs['parameter'],
    wtk_inputs['lat_lon'],wtk_inputs['year'])
wtk_data

## 3. Compare NDBC and WIND Toolkit Data

Wind conditions may be characterized by average wind speed and direction. A time series of hourly instantaneous wind speeds and directional wind rose are useful tools to visualize this information. Using the historical 10-minute continuous wind data from NDBC and WIND Toolkit wind data, we can calculate these variables using MHKiT. 

### Plot the hourly mean wind speeds on January 11, 2018
The NDBC data for January 11, 2018 is given in 10 minute intervals. We can get the instantaneous hourly wind data using the DataFrame resample() function. The WIND Toolkit data is already given as instantaneous hourly data. Then, we can plot the speeds vs the hour they occur.

In [ ]:
# Get WIND Toolkit and NDBC wind data for 2018-01-11
ndbc_hourly_data = ndbc_dict['2018']['2018-01-11'].resample('h').nearest()
wtk_hourly_data = wtk_data['2018-01-11']

# wtk_hourly_data['windspeed_10m_0']
# ndbc_hourly_data['WSPD']


# Plot the timeseries
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('Time (h)')
ax.set_ylabel('Speed (m/s)')
ax.set_title('Hourly mean wind speeds on January 11, 2018')
ax.grid()
ax.set_ylim([5, 14])
ax.set_xlim([0, 24])
line1 = ax.plot(ndbc_hourly_data.index.hour,ndbc_hourly_data['WSPD'].values,'o',label='NDBC 4m wind speed')
line2 = ax.plot(wtk_hourly_data.index.hour,wtk_hourly_data['windspeed_10m_0'].values,'x',label='WIND Toolkit 10m wind speed')
ax.legend()


### Create a mean wind speed wind rose for January 11, 2018
This exercise also considers data from January 11, 2018. We create the wind rose using the MHKiT `plot_rose` function.

In [ ]:
# Set the rose bin widths
width_direction = 10 # in degrees
width_velocity  = 1  # in m/s

# Plot the wind rose
ax = ndbc.plot_rose(ndbc_hourly_data['WDIR'],ndbc_hourly_data['WSPD'],
                     width_direction,width_velocity)


In [ ]:
ax2 = ndbc.plot_rose(wtk_data['winddirection_10m_0'],wtk_data['windspeed_10m_0'],
                     width_direction,width_velocity)

### Plot vertical temperature profile
The NDBC data for January 11, 2018 is given in 10 minute intervals. We can get the instantaneous hourly wind data using the DataFrame resample() function. The WIND Toolkit data is already given as instantaneous hourly data. Then, we can plot the speeds vs the hour they occur.